# GFS 预报数据下载 (探索)

现在我们来探索如何下载未来的天气预报数据。我们将使用 NOAA 的 GFS (Global Forecast System) 模型，这是一个全球范围、免费公开的预报系统。

**核心逻辑**:
1.  GFS 每天在 00z, 06z, 12z, 18z (UTC时间) 发布四次预报。
2.  我们需要智能地找到当前可用的、最新的那一次预报（称为“运行周期”）。
3.  根据我们关心的未来时间点（例如明天的日落），计算出相对于运行周期的“预报时效”（例如，提前24小时预报）。
4.  构建下载 URL 并获取 GRIB2 格式的数据。

In [ ]:
import logging
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# --- 1. 从我们的配置文件中导入所有常量 ---
# 因为项目是以可编辑模式安装的 (-e .), 我们可以直接从包中导入
from chromasky_toolkit import config

# --- 3. 设置日志 (这部分也可以保留在 Notebook 中) ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("GFS_Downloader")

# --- 4. 验证导入的配置 (可选) ---
print("\n--- 使用的配置 ---")
print(f"Extraction Area (North): {config.AREA_EXTRACTION['north']}")
print(f"Data Directory: {config.DATA_DIR}")

In [ ]:
# GFS 功能所需的额外库
import requests
import json
from datetime import timedelta
from typing import Tuple, Dict

def _find_latest_available_gfs_run() -> Tuple[str, str] | None:
    """
    智能判断当前可用的最新 GFS 运行周期。
    """
    logger.info("--- [GFS] 正在寻找最新的可用运行周期... ---")
    now_utc = datetime.now(timezone.utc)
    safe_margin = timedelta(hours=5)

    for i in range(10):
        potential_run_time = now_utc - timedelta(hours=i)
        run_hour = (potential_run_time.hour // 6) * 6
        run_time_utc = potential_run_time.replace(hour=run_hour, minute=0, second=0, microsecond=0)
        
        if (now_utc - run_time_utc) >= safe_margin:
            run_date_str = run_time_utc.strftime('%Y%m%d')
            run_hour_str = f"{run_time_utc.hour:02d}"
            logger.info(f"✅ 找到最新的可用运行周期: {run_date_str} {run_hour_str}z")
            return run_date_str, run_hour_str
            
    logger.error("❌ 在过去24小时内未能找到任何可用的 GFS 运行周期。")
    return None

def _get_future_target_times() -> Dict[str, datetime]:
    """根据配置，计算未来1-2天我们关心的日出日落事件的 UTC 时间。"""
    local_tz = ZoneInfo(config.LOCAL_TZ)
    now_local = datetime.now(local_tz)
    today = now_local.date()
    tomorrow = today + timedelta(days=1)
    future_events = {}
    all_times = config.SUNRISE_EVENT_TIMES + config.SUNSET_EVENT_TIMES
    for t_str in all_times:
        event_time = datetime.strptime(t_str, '%H:%M').time()
        today_event_dt = datetime.combine(today, event_time, tzinfo=local_tz)
        if today_event_dt > now_local:
            future_events[f"today_{t_str.replace(':', '')}"] = today_event_dt
        tomorrow_event_dt = datetime.combine(tomorrow, event_time, tzinfo=local_tz)
        future_events[f"tomorrow_{t_str.replace(':', '')}"] = tomorrow_event_dt
    return {name: dt.astimezone(timezone.utc) for name, dt in sorted(future_events.items())}

def download_gfs_forecast(run_date: str, run_hour: str):
    """
    为给定的 GFS 运行周期，下载所有未来目标事件的数据。
    *** 已更新为对每个时间点发起一次合并请求 ***
    """
    logger.info(f"--- [GFS] 开始为运行周期 {run_date} {run_hour}z 下载数据 ---")
    run_time_utc = datetime.strptime(f"{run_date}{run_hour}", "%Y%m%d%H").replace(tzinfo=timezone.utc)
    
    run_dir_name = f"{run_date}_t{run_hour}z"
    output_dir_base = config.GFS_DATA_DIR / run_dir_name
    output_dir_base.mkdir(parents=True, exist_ok=True)
    
    manifest_path = output_dir_base / "manifest.json"
    if manifest_path.exists():
        logger.info(f"✅ 清单文件已存在，跳过此运行周期的下载: {manifest_path}")
        return

    target_times_utc = _get_future_target_times()
    logger.info(f"将为以下 {len(target_times_utc)} 个未来事件下载数据...")

    manifest_content = {}
    for event_name, target_time in target_times_utc.items():
        time_diff_hours = (target_time - run_time_utc).total_seconds() / 3600
        forecast_hour = round(time_diff_hours)

        logger.info(f"-> 正在处理事件 '{event_name}' (预报时效: f{forecast_hour:03d})")
        
        # --- 关键修改：构建单一的合并请求 ---
        dir_param = f"/gfs.{run_date}/{run_hour}/atmos"
        file_param = f"gfs.t{run_hour}z.pgrb2.0p25.f{forecast_hour:03d}"
        
        params = {
            "file": file_param,
            "dir": dir_param,
            "subregion": "",
            "leftlon": config.AREA_EXTRACTION['west'],
            "rightlon": config.AREA_EXTRACTION['east'],
            "toplat": config.AREA_EXTRACTION['north'],
            "bottomlat": config.AREA_EXTRACTION['south'],
        }
        
        # 将所有需要的变量和层级添加到参数中
        for var in config.GFS_VARS:
            params[f"var_{var.upper()}"] = "on"
        
        req = requests.models.PreparedRequest()
        req.prepare_url(config.GFS_BASE_URL, params)
        url = req.url
        # --- 修正结束 ---

        # 文件现在是按事件保存，而不是按数据块
        event_dir = output_dir_base / f"{event_name}_f{forecast_hour:03d}"
        event_dir.mkdir(exist_ok=True)
        output_path = event_dir / "forecast_data.grib2"
        
        file_path_in_manifest = None
        try:
            logger.info(f"   发起合并下载请求...")
            response = requests.get(url, stream=True, timeout=300)
            response.raise_for_status()
            with open(output_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            logger.info(f"   ✅ 合并数据已保存到: {output_path}")
            file_path_in_manifest = str(output_path)
        except requests.exceptions.RequestException as e:
            logger.error(f"   ❌ 合并下载失败: {e}")
            logger.debug(f"   - 失败的 URL: {url}") # 打印URL以供调试
        
        manifest_content[event_name] = {
            "forecast_hour": forecast_hour,
            "target_time_utc": target_time.isoformat(),
            "file_path": file_path_in_manifest # 清单现在只记录一个文件路径
        }

    # 写入清单文件
    with open(manifest_path, 'w') as f:
        json.dump(manifest_content, f, indent=4, ensure_ascii=False)
    logger.info(f"✅ GFS 数据清单已成功写入: {manifest_path}")

In [ ]:
# --- 执行 GFS 预报数据下载 ---
gfs_run_info = _find_latest_available_gfs_run()

if gfs_run_info:
    run_date_str, run_hour_str = gfs_run_info
    download_gfs_forecast(run_date_str, run_hour_str)
else:
    logger.error("未能找到可用的 GFS 运行周期，今日预报下载任务无法执行。")